# DataPreperation

### Importiere Bibliotheken

In [1]:
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler

### Datensatz einlesen
Da Pandas die Datentypen der Merkmale nicht optimal erkennt, werden im Folgenden die konkreten Datentypen definiert. \
Diese können aus der Beschreibung des Datensatztes abgeleitet werden.

In [2]:
dtypes = {
    'UDI': 'int32',
    'Product ID': 'str',
    'Type': 'category',
    'Air temperature [K]': 'float32',
    'Process temperature [K]': 'float32',
    'Rotational speed [rpm]': 'float32',
    'Torque [Nm]': 'float32',
    'Tool wear [min]': 'float32',
    'Machine failure': 'bool',
    'TWF': 'bool',
    'HDF': 'bool',
    'PWF': 'bool',
    'OSF': 'bool',
    'RNF': 'bool'
}

df = pd.read_csv('./dataset.csv', dtype=dtypes)

### Erstelle Dummy-Merkmale für das Merkmal Maschinen-Typ

In [3]:
dummy_columns = pd.get_dummies(df['Type'], prefix='Type')
df = pd.concat([df, dummy_columns], axis=1)

### Definiere die Input- und Output- Merkmale

In [4]:
input_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Type_H', 'Type_L', 'Type_M']
output_columns = ['TWF', 'HDF', 'PWF', 'OSF', 'RNF']
numeric_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

### Implementiere Oversampling
Oversampling wird gewählt, da der Datensatz zwei sehr starke Unausgeglichenheiten enthält und ein Undersampling zu einem sehr starken Datenverlust führen würde.\
Dies liegt daran, da nur wenige Machinen tatsächlich einen Defekt aufweisen.

In [6]:
def get_label(row):
    if row['Machine failure'] == 0:
        return 'no_failure'
    else:
        for defect in output_columns:
            if row[defect] == 1:
                return defect
    return 'unknown'

df['label'] = df.apply(get_label, axis=1)

X = df[input_columns]
y = df['label']

ros = SMOTETomek(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

### Skaliere numerische Werte mit der Standartskalierung

In [7]:
scaler = StandardScaler()
df_resampled[numeric_columns] = scaler.fit_transform(df_resampled[numeric_columns])

### Speichere den vorbereiteten Datensatz

In [8]:
df_resampled.describe(include='all')
df_resampled.to_csv('./dataset_prepeared.csv')